In [1]:
import datetime

import yfinance as yf 
import investpy
import pandas as pd
import torch
import torch.nn as nn

from forexgym.utils import Query, CurrencyPair

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
end = datetime.datetime.now()
start = end - datetime.timedelta(days=1)

min_15 = yf.download('EURUSD=X', start=start, end=end, interval='15m').reset_index().drop(["Adj Close", "Volume"], axis=1)
h_1 = yf.download('EURUSD=X', start=start, end=end, interval='1h').reset_index().drop(["Adj Close", "Volume"], axis=1)

min_15["Datetime"] = pd.to_datetime(min_15["Datetime"], dayfirst=True, utc=True)
h_1["Datetime"] = pd.to_datetime(h_1["Datetime"], dayfirst=True, utc=True)

min_15.rename(columns={"Datetime": "Date"}, inplace=True)
h_1.rename(columns={"Datetime": "Date"}, inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
min_15.tail()

,Date,Open,High,Low,Close
44,2024-10-14 11:00:00+00:00,1.093135,1.093255,1.092896,1.093255
45,2024-10-14 11:15:00+00:00,1.093255,1.093255,1.093016,1.093016
46,2024-10-14 11:30:00+00:00,1.093016,1.093016,1.092896,1.092896
47,2024-10-14 11:45:00+00:00,1.092896,1.092896,1.092538,1.092657
48,2024-10-14 12:00:00+00:00,1.092657,1.092657,1.092180,1.092180


In [8]:
def select_close(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    
    return pd.DataFrame(df["Close"])

def article_processor(df: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
    df["x1"] = ((df["Close"].shift(-1) - df["Close"]) / df["Close"]).shift(1) 
    df["x2"] = ((df["High"].shift(-1) - df["High"]) / df["High"]).shift(1) 
    df["x3"] = ((df["Low"].shift(-1) - df["Low"]) / df["Low"]).shift(1) 
    df["x4"] = (df["High"] - df["Close"]) / df["Close"] 
    df["x5"] = (df["Close"] - df["Low"]) / df["Close"] 
    
    return df.drop(["Date", "Open", "High", "Low", "Close"], axis=1)

In [9]:
ticker = "EURUSD"
#timeframes = ["1m", "5m", "15m", "30m", "1H", "4H", "1D"]
timeframes = ["1H", "15m"]

query = Query(episode_length=256, trading_timeframe="15m", trading_column="Close")
# query.add_query(
#     timeframe="4H",
#     window_size=4,
#     data_processor=article_processor
# )
query.add_query(
    timeframe="1H",
    window_size=4,
    data_processor=article_processor
)
query.add_query(
    timeframe="15m",
    window_size=16,
    data_processor=article_processor
)
currency_pair = CurrencyPair(ticker, timeframes)

In [29]:
currency_pair.timeframes = {"15m": min_15, "1H": h_1}

In [30]:
data = currency_pair.generate_dataset(query, no_save=True)

Genrating EURUSD dataset...


100%|██████████| 2/2 [00:00<00:00, 173.54it/s]

Generated EURUSD dataset.


In [31]:
data.tail()

,Date,Trading_Price,1H_x1_0,1H_x2_0,1H_x3_0,1H_x4_0,1H_x5_0,1H_x1_1,1H_x2_1,1H_x3_1,...,15m_x1_14,15m_x2_14,15m_x3_14,15m_x4_14,15m_x5_14,15m_x1_15,15m_x2_15,15m_x3_15,15m_x4_15,15m_x5_15
28,2024-10-14 11:00:00+00:00,1.093255,-0.000437,-0.000656,-0.000437,0.000547,0.000109,0.000109,0.000547,0.000219,...,-0.000109,0.000000,0.000000,0.000109,0.000109,0.000000,-0.000109,0.000000,0.000000,0.000219
29,2024-10-14 11:15:00+00:00,1.093016,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,...,-0.000219,0.000000,-0.000109,0.000328,0.000000,-0.000109,0.000000,0.000000,0.000109,0.000109
30,2024-10-14 11:30:00+00:00,1.092896,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,...,0.000109,-0.000219,-0.000109,0.000000,0.000219,-0.000219,0.000000,-0.000109,0.000328,0.000000
31,2024-10-14 11:45:00+00:00,1.092657,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,...,-0.000109,0.000328,0.000109,0.000437,0.000000,0.000109,-0.000219,-0.000109,0.000000,0.000219
32,2024-10-14 12:00:00+00:00,1.092180,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,...,0.000328,0.000000,0.000000,0.000109,0.000328,-0.000109,0.000328,0.000109,0.000437,0.000000


In [32]:
data = data.drop(["Date", "Trading_Price"], axis=1)

In [13]:
class Network(nn.Module):
    def __init__(self, n_inputs: int, n_outputs: int, hidden_size: int = 128, n_layers: int = 2):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, hidden_size),
            *[
                nn.ReLU() if i % 2 == 0 else nn.Linear(hidden_size, hidden_size) for i in range(n_layers*2)
            ],
            nn.ReLU(),
            nn.Linear(hidden_size, n_outputs),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.softmax(self.layers(x))

In [14]:
model = torch.load("models/supervised_02.pt", weights_only=False)
model.eval()

Network(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=2, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [35]:
features.shape

torch.Size([3, 100])

In [34]:
features = torch.from_numpy(data.iloc[-3:].to_numpy()).to(device).to(dtype=torch.float32)


In [36]:
data.tail()

,1H_x1_0,1H_x2_0,1H_x3_0,1H_x4_0,1H_x5_0,1H_x1_1,1H_x2_1,1H_x3_1,1H_x4_1,1H_x5_1,...,15m_x1_14,15m_x2_14,15m_x3_14,15m_x4_14,15m_x5_14,15m_x1_15,15m_x2_15,15m_x3_15,15m_x4_15,15m_x5_15
28,-0.000437,-0.000656,-0.000437,0.000547,0.000109,0.000109,0.000547,0.000219,0.000766,0.000109,...,-0.000109,0.000000,0.000000,0.000109,0.000109,0.000000,-0.000109,0.000000,0.000000,0.000219
29,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,-0.000219,0.000000,-0.000109,0.000328,0.000000,-0.000109,0.000000,0.000000,0.000109,0.000109
30,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,0.000109,-0.000219,-0.000109,0.000000,0.000219,-0.000219,0.000000,-0.000109,0.000328,0.000000
31,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,-0.000109,0.000328,0.000109,0.000437,0.000000,0.000109,-0.000219,-0.000109,0.000000,0.000219
32,-0.000328,-0.000546,-0.000328,0.000328,0.000109,-0.000437,-0.000656,-0.000437,0.000547,0.000109,...,0.000328,0.000000,0.000000,0.000109,0.000328,-0.000109,0.000328,0.000109,0.000437,0.000000


In [37]:
model.forward(features)

tensor([[0.9932, 0.0068],
        [0.8571, 0.1429],
        [0.0063, 0.9937]], device='cuda:0', grad_fn=<SoftmaxBackward0>)